In [7]:
import numpy as np
import os
import tensorflow as tf
from scipy.ndimage import rotate
import cv2
import matplotlib.pyplot as plt


In [12]:
data = np.load('/Users/annabelcourt/code/ladis-jil/Predicting-Economic-Activity-Satelite-Imagery/raw_data/day_images_image_0.npz')

In [13]:
print(data)

NpzFile '/Users/annabelcourt/code/ladis-jil/Predicting-Economic-Activity-Satelite-Imagery/raw_data/day_images_image_0.npz' with keys: arr_0, arr_1, arr_2, arr_3, arr_4...


In [8]:
def convert_npz_to_bgr(npz_file_path):
    # Load the .npz file
    data = np.load(npz_file_path)
    bgr_images = {}
    # Iterate over all keys in the file
    for key in data.keys():
        # Extract the image data
        image = data[key]
        image = np.expand_dims(image, axis=0)
        # Check if the image needs to be resized
        if image.shape[0] != 224 or image.shape[1] != 224:
            image = tf.image.resize(image, (224, 224))
        # Convert the image from RGB to BGR (if necessary)
        # VGG16 expects the image in BGR format.
        if image.shape[2] == 3:  # Check if the image has 3 color channels
            image = tf.reverse(image, axis=[-1])  # Reversing channels from RGB to BGR
        # Ensure the image is in the shape (224, 224, 3)
        if image.shape != (224, 224, 3):
            raise ValueError(f"Image shape is {image.shape}, expected (224, 224, 3)")
        # Convert the image to float32, as VGG16 expects inputs in float32
        image = tf.cast(image, tf.float32)
        # Preprocess the image using VGG16's preprocessing function
        image = tf.keras.applications.vgg16.preprocess_input(image)
        # Store the preprocessed image
        bgr_images[key] = image
    return bgr_images

# Example usage
npz_file_path = '/Users/annabelcourt/code/ladis-jil/Predicting-Economic-Activity-Satelite-Imagery/raw_data/day_images_image_0.npz'
bgr_images = convert_npz_to_bgr(npz_file_path)
for key, image in bgr_images.items():
    print(f"Key: {key}, Image shape: {image.shape}")

Key: arr_0, Image shape: (224, 224, 3)
Key: arr_1, Image shape: (224, 224, 3)
Key: arr_2, Image shape: (224, 224, 3)
Key: arr_3, Image shape: (224, 224, 3)
Key: arr_4, Image shape: (224, 224, 3)
Key: arr_5, Image shape: (224, 224, 3)
Key: arr_6, Image shape: (224, 224, 3)
Key: arr_7, Image shape: (224, 224, 3)
Key: arr_8, Image shape: (224, 224, 3)
Key: arr_9, Image shape: (224, 224, 3)
Key: arr_10, Image shape: (224, 224, 3)
Key: arr_11, Image shape: (224, 224, 3)
Key: arr_12, Image shape: (224, 224, 3)
Key: arr_13, Image shape: (224, 224, 3)
Key: arr_14, Image shape: (224, 224, 3)
Key: arr_15, Image shape: (224, 224, 3)
Key: arr_16, Image shape: (224, 224, 3)
Key: arr_17, Image shape: (224, 224, 3)
Key: arr_18, Image shape: (224, 224, 3)
Key: arr_19, Image shape: (224, 224, 3)
Key: arr_20, Image shape: (224, 224, 3)
Key: arr_21, Image shape: (224, 224, 3)
Key: arr_22, Image shape: (224, 224, 3)
Key: arr_23, Image shape: (224, 224, 3)
Key: arr_24, Image shape: (224, 224, 3)
Key: arr_2

In [9]:
def visualize_bgr_images(bgr_images):
    for key, image in bgr_images.items():
        # Convert the image back from BGR to RGB
        rgb_image = tf.reverse(image, axis=[-1]).numpy()
        # Reverse VGG16 preprocessing
        rgb_image = rgb_image[..., ::-1]  # Reverse BGR to RGB
        rgb_image += [103.939, 116.779, 123.68]  # Add VGG16 mean values
        rgb_image = np.clip(rgb_image, 0, 255).astype('uint8')
        
        # Display the image using Matplotlib
        plt.imshow(rgb_image)
        plt.title(key)
        plt.axis('off')
        plt.show()

In [ ]:
visualize_bgr_images(bgr_images)

In [15]:
def crop_resize(tensor, crop_size=23):
    """
    Randomly crops the input tensor.
    Args:
        tensor (tf.Tensor): Input tensor.
        crop_size (int): Size of the cropped region (201 is 90% of 224 so crop by 23).
    Returns:
        tf.Tensor: Cropped tensor with size 224,224.
    """
    height, width, _ = tensor.shape
    x = np.random.randint(0, width - crop_size)
    y = np.random.randint(0, height - crop_size)
    cropped_tensor = tf.image.crop_to_bounding_box(tensor, y, x, crop_size, crop_size)
    cropped_resized_tensor = tf.image.resize(cropped_tensor, (224, 224))
    return cropped_resized_tensor
# will the size be 224,224,3 or just 224,224?

In [ ]:

def rotate_90(tensor):
    tf.image.rot90(tensor, k=1)
    return rotated_90_tensor

def rotate_180(tensor):
    tf.image.rot90(tensor, k=2)
    return rotated_180_tensor

def rotate_270(tensor):
    tf.image.rot90(tensor, k=3)
    return rotated_270_tensor

def random_flip(tensor):
    """
    Randomly flips the input tensor horizontally or vertically.
    Args:
        tensor (tf.Tensor): Input tensor.
    Returns:
        tf.Tensor: Flipped tensor.
    """
    if np.random.rand() < 0.5:
        flipped_tensor = tf.image.flip_left_right(tensor)  # Horizontal flip
    else:
        flipped_tensor = tf.image.flip_up_down(tensor)  # Vertical flip
    return flipped_tensor


def augment_tensors(tensors_dict, output_dir, num_augmentations=5):
    """
    Augments a dictionary of tensors using cropping, rotation, and different flips.
    Args:
        tensors_dict (dict of tf.Tensor): Dictionary of input tensors.
        output_dir (str): Directory to save the augmented tensors.
        num_augmentations (int): Number of augmentations to create for each tensor (default is 5).
    Returns:
        None
    """
    os.makedirs(output_dir, exist_ok=True)
    for key, tensor in tensors_dict.items():  # Iterate over dictionary items
        for i in range(num_augmentations):
            cropped_tensor = 90_crop_resize(tensor)
            rotated_tensor = random_rotation(cropped_tensor)
            flipped_tensor = random_flip(rotated_tensor)
            output_path = os.path.join(output_dir, f"augmented_tensor_{key}_{i}.png")
            flipped_array = flipped_tensor.numpy()
            cv2.imwrite(output_path, (flipped_array * 255).astype(np.uint8))
            print(f"Augmented tensor {key} version {i} saved at {output_path}")

# Example usage
input_tensors = bgr_images  # Assuming bgr_images is a dictionary of tensors
output_directory = 'augmented_daytime_images_test_0'
augment_tensors(input_tensors, output_directory)


input_tensors =  bgr_images
output_directory = 'augmented_daytime_images_test_0'
augment_tensors(input_tensors, output_directory)

In [4]:
print(bgr_images)

{'arr_0': <tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[-103.38214 , -116.3157  , -123.319214],
        [-103.36533 , -116.29244 , -123.30241 ],
        [-103.2925  , -116.222694, -123.22958 ],
        ...,
        [-103.32892 , -116.247345, -123.250305],
        [-103.36141 , -116.27984 , -123.279724],
        [-103.391945, -116.31037 , -123.30941 ]],

       [[-103.38214 , -116.3157  , -123.319214],
        [-103.36533 , -116.29244 , -123.30241 ],
        [-103.2925  , -116.222694, -123.22958 ],
        ...,
        [-103.32892 , -116.247345, -123.250305],
        [-103.36141 , -116.27984 , -123.279724],
        [-103.391945, -116.31037 , -123.30941 ]],

       [[-103.38214 , -116.3157  , -123.319214],
        [-103.36533 , -116.29244 , -123.30241 ],
        [-103.2925  , -116.222694, -123.22958 ],
        ...,
        [-103.32892 , -116.247345, -123.250305],
        [-103.36141 , -116.27984 , -123.279724],
        [-103.391945, -116.31037 , -123.30941 ]],

       .